In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sortedcontainers import SortedKeyList

In [82]:
    data_file = "demo-data.csv"
    data = pd.read_csv(data_file)
    data['ID'] = data['ID'].astype('int')

In [62]:
class RatchetNode:

    def __init__(self, id: int, features: np.ndarray):
        self.id = int(id)
        self.features = np.array(features)
        self.mag = magnitude(self.features)
        self.enclosed = None
        self.extra = None

    def __repr__(self):
        
        return f'RatchetNode({self.id}, {self.features}, {self.enclosed})'

    def __str__(self):
        return self.__repr__()

    def get_feature(self, idx: int):
        return self.features[idx]

    def get_mag(self):
        return self._mag

Chebychev distance from origin

In [27]:
def magnitude(arr: np.ndarray):
    return arr.max()

In [63]:
queue = SortedKeyList(key=lambda node: node.mag)

In [64]:
for id, row in data.iterrows():
    id = row[0]
    features = list(row[1:])
    rn = RatchetNode(id, np.array(features))
    queue.add(rn)

# Sort and scan method

1. Create list sorted by chebyshev distance
1. For each item, compute size of "contained" set by scanning earlier items in the list in inverse order
    1. If a "candidate" item or "extra" is encountered, stop 
1. If an item contains exactly k points, mark as "candidate"
1. If an item contains more than k points, mark as "extra"

In [57]:
def encloses(p1: np.ndarray, p2: np.ndarray):
    return np.sum(p1 < p2) == 0

def encloses_node(n1: RatchetNode, n2: RatchetNode):
    return encloses(n1.features, n2.features)

In [58]:
def scan_left(i):
    while i > 0:
        node = queue[i-1]
        yield node
        i -= 1


In [72]:
candidate_limit = 20

def process_node(i):
    rn = queue[i]
    scanner = scan_left(i)
    enclosed_count = 0
    
    try:
        while True:
            this_node = next(scanner)
            #if this_node.enclosed == candidate_limit or this_node.extra == True:
            #    rn.enclosed = -1
            #    rn.extra = True
            #    break
            if encloses_node(rn, this_node):
                enclosed_count += 1
    except StopIteration:
        rn.enclosed = enclosed_count
     

In [73]:
for i in range(0, len(queue)):
    process_node(i)

In [77]:
candidates = [node for node in queue if node.enclosed == candidate_limit]

In [78]:
def list_enclosed(rn):
    return [node for node in queue if encloses_node(rn, node)]

In [79]:
candidates

[RatchetNode(91, [0.51051053 0.6026692  0.70199911], 20)]

In [80]:
list_enclosed(candidates[0])

[RatchetNode(55, [0.14928399 0.08101213 0.01935286], 0),
 RatchetNode(71, [0.16107311 0.05933266 0.19983314], 0),
 RatchetNode(13, [0.21374091 0.2069472  0.06968971], 1),
 RatchetNode(92, [0.16703373 0.22335874 0.11206233], 1),
 RatchetNode(84, [0.20410631 0.24104136 0.2200862 ], 3),
 RatchetNode(22, [0.1913797  0.25830743 0.19099136], 2),
 RatchetNode(76, [0.23200753 0.01366697 0.29731575], 0),
 RatchetNode(20, [0.30034798 0.13201381 0.15929502], 1),
 RatchetNode(79, [0.32155202 0.22347597 0.02227091], 1),
 RatchetNode(86, [0.36313654 0.14224904 0.01478904], 0),
 RatchetNode(18, [0.37558674 0.28444592 0.37601546], 10),
 RatchetNode(74, [0.27281377 0.43706438 0.15830621], 3),
 RatchetNode(51, [0.4405583  0.44693552 0.3037832 ], 11),
 RatchetNode(45, [0.19296336 0.03566249 0.44949457], 0),
 RatchetNode(29, [0.4594202  0.13714224 0.2813975 ], 3),
 RatchetNode(72, [0.3131179  0.21289314 0.49027967], 6),
 RatchetNode(80, [0.44229574 0.33841391 0.54574921], 13),
 RatchetNode(52, [0.03105185